# JS: Scroll Navigation

> JavaScript generator for scroll-to-nav conversion.

In [ ]:
#| default_exp js.scroll

In [ ]:
#| export
from typing import Tuple

from cjm_fasthtml_card_stack.core.html_ids import CardStackHtmlIds
from cjm_fasthtml_card_stack.core.button_ids import CardStackButtonIds
from cjm_fasthtml_card_stack.core.constants import SCROLL_THRESHOLD, NAVIGATION_COOLDOWN

## generate_scroll_nav_js

Converts mouse wheel events on the card stack container into navigation
button clicks. Uses delta accumulation with a cooldown to handle both
trackpads and scroll wheels naturally. Supports mode-based disabling.

In [ ]:
#| export
def generate_scroll_nav_js(
    ids: CardStackHtmlIds,  # HTML IDs for this card stack instance
    button_ids: CardStackButtonIds,  # Button IDs for navigation triggers
    disable_in_modes: Tuple[str, ...] = (),  # Mode names where scroll nav is suppressed
) -> str:  # JavaScript code fragment for scroll navigation
    """Generate JS for scroll wheel to navigation conversion."""
    # Build mode check
    if disable_in_modes:
        modes_array = ', '.join(f"'{m}'" for m in disable_in_modes)
        mode_check = f"""
        function isScrollDisabled() {{
            if (typeof window.kbNav !== 'undefined') {{
                const state = window.kbNav.getState();
                const disabledModes = [{modes_array}];
                return state && disabledModes.includes(state.currentMode);
            }}
            return false;
        }}
        """
        mode_guard = "if (isScrollDisabled()) return;"
    else:
        mode_check = ""
        mode_guard = ""

    return f"""
        // === Scroll Navigation ===
        const _scrollState = {{ accumulatedDelta: 0, lastNavTime: 0 }};
        const _SCROLL_THRESHOLD = {SCROLL_THRESHOLD};
        const _NAV_COOLDOWN = {NAVIGATION_COOLDOWN};
        {mode_check}
        function setupScrollNavigation() {{
            const cardStack = document.getElementById('{ids.card_stack}');
            if (!cardStack || cardStack._scrollNavSetup) return;
            cardStack._scrollNavSetup = true;

            cardStack.addEventListener('wheel', function(evt) {{
                {mode_guard}
                evt.preventDefault();

                const now = Date.now();
                if (now - _scrollState.lastNavTime > _NAV_COOLDOWN * 2) {{
                    _scrollState.accumulatedDelta = 0;
                }}
                _scrollState.accumulatedDelta += evt.deltaY;

                if (Math.abs(_scrollState.accumulatedDelta) >= _SCROLL_THRESHOLD) {{
                    if (now - _scrollState.lastNavTime >= _NAV_COOLDOWN) {{
                        const btnId = _scrollState.accumulatedDelta > 0
                            ? '{button_ids.nav_down}' : '{button_ids.nav_up}';
                        _scrollState.accumulatedDelta = 0;
                        _scrollState.lastNavTime = now;
                        const btn = document.getElementById(btnId);
                        if (btn) btn.click();
                    }}
                }}
            }}, {{ passive: false }});
        }}

        // Expose for master coordinator to re-setup after swaps
        ns._setupScrollNav = setupScrollNavigation;
    """

In [ ]:
# Test scroll nav JS generation
ids = CardStackHtmlIds(prefix="cs0")
btn = CardStackButtonIds(prefix="cs0")
js = generate_scroll_nav_js(ids, btn)
assert ids.card_stack in js
assert btn.nav_up in js
assert btn.nav_down in js
assert "isScrollDisabled" not in js  # No mode check when no modes
assert "ns._setupScrollNav" in js
print("Scroll nav JS basic tests passed!")

Scroll nav JS basic tests passed!


In [ ]:
# Test with disabled modes
js_modes = generate_scroll_nav_js(ids, btn, disable_in_modes=("split", "edit"))
assert "isScrollDisabled" in js_modes
assert "'split'" in js_modes
assert "'edit'" in js_modes
print("Scroll nav JS mode disabling tests passed!")

Scroll nav JS mode disabling tests passed!


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()